In [1]:
# Library load
# Basic libraries
import time
import json
import requests  # for function calls like get_stock_price, if they use external
import warnings
import os
import requests
import asyncio
import configparser
import asyncio
# data analysis
import numpy as np
import pandas as pd
# financial libs
import yfinance as yf # terminal install -> pip install yfinance
import talib # terminal install -> pip install TA-Lib
# OpenAI libs
import openai # terminal install ->  pip install openai
from openai import OpenAI  
from agents import Runner, Agent,function_tool, items # terminal install -> pip 
from agents import (Agent, Runner, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    handoff,  InputGuardrailTripwireTriggered, RunConfig, ModelSettings)
from agents.exceptions import InputGuardrailTripwireTriggered
from agents.extensions import handoff_filters
from pydantic import BaseModel

In [2]:
## Retrieve API Keys and start OpenAI session
#  retrieve keys:
key_file_name = 'api_key'
path = '/mnt/sda1/PythonProject/LLM_TRADE'  # use your path to where you
config = configparser.ConfigParser()
config.read(path+'/'+ key_file_name) 
api_key = config['openai']['api_key'] 
os.environ["OPENAI_API_KEY"] = api_key # required by OpenAI agents
fmp_key = config['financialmodelingprep']['api_key'] 
warnings.filterwarnings('ignore')

In [3]:
# LLMs models available for the Agent in OpenAI
models = openai.models.list()
for model in models.to_dict()['data']:
    print(model['id'])




text-embedding-ada-002
whisper-1
gpt-3.5-turbo
tts-1
gpt-3.5-turbo-16k
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
o1-preview-2024-09-12
o1-preview
o1-mini-2024-09-12
o1-mini
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-audio-preview
gpt-4o-2024-11-20
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-image-1


In [4]:
import requests
import numpy as np
import talib

# Replace with your actual FMP API key

def get_historical_data(symbol: str, apikey: str) -> np.ndarray:
    """
    Fetch historical closing prices from FMP API.
    Returns a numpy array of closing prices, oldest to most recent.
    """
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={apikey}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an exception for 4xx/5xx errors
        data = response.json()
        if "historical" not in data or not data["historical"]:
            return np.array([])
        # Reverse to get oldest first, as FMP returns most recent first
        closes = [day["close"] for day in data["historical"][::-1]]
        return np.array(closes, dtype='float64')
    except Exception as e:
        print(f"Error fetching historical data for {symbol}: {e}")
        return np.array([])

def get_stock_company_info(symbol: str, apikey: str) -> dict:
    """
    Fetch company's basic information using FMP API.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={apikey}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        info = response.json()
        if not info:
            return {"error": f"No data found for symbol: {symbol}"}
        return {
            "symbol": symbol.upper(),
            "price": info[0]["price"],
            "beta": info[0]["beta"],
            "description": info[0]["description"],
            "sector": info[0]["sector"],
            "industry": info[0]["industry"],
            "dcf_valuation": info[0]["dcf"]
        }
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

def get_stock_technical(symbol: str, apikey: str) -> dict:
    """
    Fetches technical indicators using historical data from FMP API.
    """
    try:
        close_np = get_historical_data(symbol, apikey)
        if len(close_np) == 0:
            return {"error": f"No historical data for symbol: {symbol}"}
        if len(close_np) < 200:
            return {"error": f"Not enough data to compute indicators for {symbol} (need at least 200 days)"}
        
        # Compute indicators
        sma_50 = talib.SMA(close_np, timeperiod=50)
        sma_200 = talib.SMA(close_np, timeperiod=200)
        rsi = talib.RSI(close_np, timeperiod=14)
        
        latest_sma_50 = sma_50[-1]
        latest_sma_200 = sma_200[-1]
        latest_rsi = rsi[-1]
        last_price = close_np[-1]
        
        # Calculate 6-month momentum (approx. 126 trading days)
        if len(close_np) >= 127:  # Need 127 elements to get 126 days ago
            roc_6m = close_np[-1] / close_np[-127] - 1
        else:
            roc_6m = None
        
        if np.isnan(latest_sma_50) or np.isnan(latest_sma_200) or np.isnan(latest_rsi):
            return {"error": f"Indicator values not ready — possible NaNs at the end"}
        
        return {
            "symbol": symbol.upper(),
            "last_price": round(last_price, 2),
            "SMA_50d": round(latest_sma_50, 2),
            "SMA_200d": round(latest_sma_200, 2),
            "RSI": round(latest_rsi, 2),
            "6_month_price_momentum": round(roc_6m, 2) if roc_6m is not None else None
        }
    except Exception as e:
        return {"error": f"Unhandled error for symbol {symbol}: {str(e)}"}

# Test the functions
if __name__ == "__main__":
    symbol = "NVDA"
    print(f"Fetching data for {symbol}...")
    company_info = get_stock_company_info(symbol, fmp_key)
    technical_info = get_stock_technical(symbol, fmp_key)
    print("\nCompany Info:", company_info)
    print("\nTechnical Info:", technical_info)

Fetching data for NVDA...

Company Info: {'symbol': 'NVDA', 'price': 144.12, 'beta': 2.122, 'description': "NVIDIA Corporation provides graphics, and compute and networking solutions in the United States, Taiwan, China, and internationally. The company's Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building 3D designs and virtual worlds. Its Compute & Networking segment provides Data Center platforms and systems for AI, HPC, and accelerated computing; Mellanox networking and interconnect solutions; automotive AI Cockpit, autonomous driving development agreements, and autonomous vehicle solutions; cryptocurrency mining processors; Jetson for robotics and other embedded platforms; and

In [ ]:
import asyncio
import requests
import yfinance as yf
import talib
import numpy as np
from openai import OpenAI



# ---- 1. Define Tools ----
# ---- 1. Define Tools ----
def get_historical_data(symbol: str) -> np.ndarray:
    """
    Fetch historical closing prices from FMP API.
    Returns a numpy array of closing prices, oldest to most recent.
    """
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={fmp_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an exception for 4xx/5xx errors
        data = response.json()
        if "historical" not in data or not data["historical"]:
            return np.array([])
        # Reverse to get oldest first, as FMP returns most recent first
        closes = [day["close"] for day in data["historical"][::-1]]
        return np.array(closes, dtype='float64')
    except Exception as e:
        print(f"Error fetching historical data for {symbol}: {e}")
        return np.array([])

@function_tool
def get_stock_company_info(symbol: str) -> dict:
    """
    Fetch company's basic information using FMP API.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={fmp_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        info = response.json()
        if not info:
            return {"error": f"No data found for symbol: {symbol}"}
        return {
            "symbol": symbol.upper(),
            "price": info[0]["price"],
            "beta": info[0]["beta"],
            "description": info[0]["description"],
            "sector": info[0]["sector"],
            "industry": info[0]["industry"],
            "dcf_valuation": info[0]["dcf"]
        }
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

@function_tool
def get_stock_technical(symbol: str) -> dict:
    """
    Fetches technical indicators using historical data from FMP API.
    """
    try:
        close_np = get_historical_data(symbol)
        if len(close_np) == 0:
            return {"error": f"No historical data for symbol: {symbol}"}
        if len(close_np) < 200:
            return {"error": f"Not enough data to compute indicators for {symbol} (need at least 200 days)"}
        
        # Compute indicators
        sma_50 = talib.SMA(close_np, timeperiod=50)
        sma_200 = talib.SMA(close_np, timeperiod=200)
        rsi = talib.RSI(close_np, timeperiod=14)
        
        latest_sma_50 = sma_50[-1]
        latest_sma_200 = sma_200[-1]
        latest_rsi = rsi[-1]
        last_price = close_np[-1]
        
        # Calculate 6-month momentum (approx. 126 trading days)
        if len(close_np) >= 127:  # Need 127 elements to get 126 days ago
            roc_6m = close_np[-1] / close_np[-127] - 1
        else:
            roc_6m = None
        
        if np.isnan(latest_sma_50) or np.isnan(latest_sma_200) or np.isnan(latest_rsi):
            return {"error": f"Indicator values not ready — possible NaNs at the end"}
        
        return {
            "symbol": symbol.upper(),
            "last_price": round(last_price, 2),
            "SMA_50d": round(latest_sma_50, 2),
            "SMA_200d": round(latest_sma_200, 2),
            "RSI": round(latest_rsi, 2),
            "6_month_price_momentum": round(roc_6m, 2) if roc_6m is not None else None
        }
    except Exception as e:
        return {"error": f"Unhandled error for symbol {symbol}: {str(e)}"}

# ---- 2. Guardrail: block crypto and invalid symbols ----
class GuardrailOutput(BaseModel):
    allow: bool
    reason: str

crypto_keywords = {"bitcoin", "btc", "eth", "ethereum", "crypto", "doge", "solana"}

async def reject_invalid_tickers(ctx, agent, input_data):
    input_text = input_data.lower()
    if any(word in input_text for word in crypto_keywords):
        return GuardrailFunctionOutput(
            output_info=GuardrailOutput(allow=False, reason="Crypto queries are not supported."),
            tripwire_triggered=True
        )
    return GuardrailFunctionOutput(
        output_info=GuardrailOutput(allow=True, reason="OK"),
        tripwire_triggered=False
    )

# ---- 3. Compliance Agent ----
compliance_agent = Agent(
    name="Compliance Agent",
    instructions="You respond to questions that require regulatory caution. Do not provide legal advice but offer general guidance on compliance-related topics."
)

compliance_handoff = handoff(
    agent=compliance_agent,
    input_filter=handoff_filters.remove_all_tools  # erase any previous chat info
)

# ---- 4. Trading Assistant Agent ----
trading_assistant = Agent(
    name="trading_assistant",
    instructions="""
You are a stock trading assistant focused on providing accurate and concise information about stocks. Your responsibilities include:

- Using the provided tools to fetch and summarize stock data.
- Reporting results in a clear, markdown-formatted manner, using tables for numerical data.
- Providing BUY, SELL, or HOLD recommendations only when explicitly requested, based solely on the retrieved data.
- Escalating any legal or regulatory questions to the Compliance Agent.
- Refraining from answering non-investment-related queries.

Rules and constraints:
- Always use the tools to obtain real-time or provided data; never guess or assume information.
- Do not include disclaimers or additional context unless specifically asked.
- Keep responses focused and avoid off-topic elaboration.
- If a tool fails or returns no data, clearly state this without making assumptions.
- Use markdown formatting: bold for headers, code formatting for ticker symbols, and tables for indicators.

Remember, your role is to assist with stock trading decisions based on data, not to provide general advice or opinions.
""",
    tools=[get_stock_company_info, get_stock_technical],
    input_guardrails=[InputGuardrail(guardrail_function=reject_invalid_tickers)],
    handoffs=[compliance_handoff],
    handoff_description="Compliance_agent",
    model = "gpt-4o-mini"
)

# ---- 5. Workflow Simulation ----
config = RunConfig(
    model_settings=ModelSettings(
        temperature=0.2  # Lower temp = more focused, less randomness in responses
    )
)

async def main():
    output = []  # store each query answer
    queries = [
        "What's the price trend of NViDA?",
        "What's your view on BTC?",  # crypto question
        "What's the valuation of Nvidia?",
        "What are the implications of EU GDPR for an asset manager?",  # regulatory question
        "What's the market sensitivity of NVDA aka beta?",  # investment question
        "What is a better investment NVDA or AMD?",  # investment question
        "What is life?"  # non-investment question
    ]
    for i, query in enumerate(queries):
        try:
            print(f'Q_{i+1}: {query}')
            result = await Runner.run(trading_assistant, query, run_config=config)
            print(f'A: {result.final_output}')
            output.append(result)
        except InputGuardrailTripwireTriggered as e:
            reason = e.guardrail_result.output.output_info.reason
            print(f"Guardrail blocked input: {reason}")
            continue
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            continue
        print("\n", 175 * "-", "\n")
    return queries, output

# Execute Workflow Simulation:
try:
    queries, result_list = await main()
except RuntimeError:
    asyncio.run(main())

Error getting response: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-Vulk4ceYNK1VBMPLuIceq3Md on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}. (request_id: req_c9878d96739a3749763993e5b9cc0f79)


In [16]:

try:
    from darts import TimeSeries
    from darts.models import NBEATSModel
except ImportError as e:
    print(f"Failed to import Darts: {e}")
    raise
import nest_asyncio

# Apply nest_asyncio for Jupyter compatibility
nest_asyncio.apply()

# FMP API key (replace with your actual key)

# Tool to fetch historical data
def get_historical_data(symbol: str) -> pd.DataFrame:
    """
    Fetch historical closing prices from FMP API.
    Returns DataFrame with 'date' and 'close' columns, ensuring daily frequency.
    """
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={fmp_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if "historical" not in data or not data["historical"]:
            return pd.DataFrame()
        df = pd.DataFrame(data["historical"])[["date", "close"]]
        df["date"] = pd.to_datetime(df["date"])
        # Set date as index
        df.set_index("date", inplace=True)
        # Create a complete date range (daily, including weekends)
        date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
        # Reindex to fill missing dates, forward-filling prices
        df = df.reindex(date_range, method='ffill')
        # Reset index to have 'date' as a column
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'date'}, inplace=True)
        return df
    except Exception as e:
        print(f"Error fetching historical data for {symbol}: {e}")
        return pd.DataFrame()

@function_tool
def get_stock_company_info(symbol: str) -> dict:
    """
    Fetch company's basic information using FMP API.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={fmp_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        info = response.json()
        if not info:
            return {"error": f"No data found for symbol: {symbol}"}
        return {
            "symbol": symbol.upper(),
            "price": info[0]["price"],
            "beta": info[0]["beta"],
            "description": info[0]["description"],
            "sector": info[0]["sector"],
            "industry": info[0]["industry"],
            "dcf_valuation": info[0]["dcf"]
        }
    except Exception as e:
        return {"error": f"API request failed: {str(e)}"}

@function_tool
def get_stock_technical(symbol: str) -> dict:
    """
    Calculate technical indicators (SMA, RSI, MACD) from FMP API historical data.
    """
    try:
        df = get_historical_data(symbol)
        if df.empty:
            return {"error": f"No historical data for symbol: {symbol}"}
        close_np = df["close"].to_numpy(dtype='float64')
        if len(close_np) < 200:
            return {"error": f"Not enough data to compute indicators for {symbol} (need at least 200 days)"}
        
        # Calculate indicators
        sma_50 = talib.SMA(close_np, timeperiod=50)
        sma_200 = talib.SMA(close_np, timeperiod=200)
        rsi = talib.RSI(close_np, timeperiod=14)
        macd, macdsignal, macdhist = talib.MACD(close_np, fastperiod=12, slowperiod=26, signalperiod=9)
        
        latest_sma_50 = sma_50[-1]
        latest_sma_200 = sma_200[-1]
        latest_rsi = rsi[-1]
        last_price = close_np[-1]
        latest_macd = macd[-1]
        latest_macdsignal = macdsignal[-1]
        latest_macdhist = macdhist[-1]
        
        # Calculate 6-month momentum
        if len(close_np) >= 126:  # Approx. 6 months
            roc_6m = close_np[-1] / close_np[-126] - 1
        else:
            roc_6m = None
        
        if np.isnan(latest_sma_50) or np.isnan(latest_sma_200) or np.isnan(latest_rsi) or np.isnan(latest_macd):
            return {"error": f"Indicator values not ready"}
        
        return {
            "symbol": symbol.upper(),
            "last_price": round(last_price, 2),
            "SMA_50d": round(latest_sma_50, 2),
            "SMA_200d": round(latest_sma_200, 2),
            "RSI": round(latest_rsi, 2),
            "MACD": round(latest_macd, 2),
            "MACD_signal": round(latest_macdsignal, 2),
            "MACD_hist": round(latest_macdhist, 2),
            "6_month_momentum": round(roc_6m, 2) if roc_6m is not None else None
        }
    except Exception as e:
        return {"error": f"Unhandled error for symbol {symbol}: {str(e)}"}

@function_tool
def get_stock_forecast(symbol: str, horizon: int = 20) -> dict:
    """
    Forecast stock prices for 'horizon' days using N-BEATS model.
    """
    try:
        df = get_historical_data(symbol)
        if df.empty:
            return {"error": f"No data available for symbol: {symbol}"}
        
        # Create TimeSeries with explicit frequency
        series = TimeSeries.from_dataframe(
            df,
            time_col='date',
            value_cols='f',
            fill_missing_dates=True,
            freq='D'  # Daily frequency
        )
        
        # Split into training and validation sets
        train, _ = series.split_after(0.8)
        
        # Initialize and train N-BEATS model
        model = NBEATSModel(input_chunk_length=30, output_chunk_length=horizon)
        model.fit(train, epochs=5, verbose=False)  # Reduced epochs for speed
        
        # Forecast
        forecast = model.predict(n=horizon)
        predicted_prices = forecast.values().flatten().tolist()
        
        return {
            "symbol": symbol.upper(),
            "forecast": [round(price, 2) for price in predicted_prices]
        }
    except Exception as e:
        return {"error": f"Forecasting failed for {symbol}: {str(e)}"}

# Guardrail: block crypto
class GuardrailOutput(BaseModel):
    allow: bool
    reason: str

crypto_keywords = {"bitcoin", "btc", "crypto", "doge", "solana", "eth", "ethereum"}

async def reject_invalid_tickers(ctx, agent, input_data):
    input_text = input_data.lower()
    if any(word in input_text for word in crypto_keywords):
        return GuardrailFunctionOutput(
            output_info=GuardrailOutput(allow=False, reason="Crypto queries are not supported."),
            tripwire_triggered=True
        )
    return GuardrailFunctionOutput(
        output_info=GuardrailOutput(allow=True, reason="OK"),
        tripwire_triggered=False
    )

# Compliance Agent
compliance_agent = Agent(
    name="Compliance Agent",
    instructions="You respond to questions requiring regulatory caution. Do not provide legal advice but offer general guidance on compliance-related topics."
)

compliance_handoff = handoff(
    agent=compliance_agent,
    input_filter=handoff_filters.remove_all_tools
)

# Trading Assistant Agent
trading_assistant = Agent(
    name="Trading Assistant",
    instructions="""
You are a stock trading assistant focused on providing accurate and concise stock information. Your responsibilities include:

- Using provided tools to fetch and summarize stock data.
- Reporting results in clear markdown format, using tables for numerical data.
- Providing BUY, SELL, or HOLD recommendations only when explicitly requested, based solely on retrieved data.
- Escalating legal or regulatory questions to the Compliance Agent.
- Refraining from answering non-investment-related queries.
- Using get_stock_forecast for future price predictions when requested.

Rules and constraints:
- Always use tools for real-time or provided data; never guess or assume information.
- Do not include disclaimers or extra context unless asked.
- If a tool fails or no data is returned, state this clearly without assumptions.
- Use markdown: bold headers, code formatting for tickers, tables for indicators.

Your role is to assist with data-driven stock trading decisions, not to provide general advice or opinions.
""",
    tools=[get_stock_company_info, get_stock_technical, get_stock_forecast],
    input_guardrails=[InputGuardrail(guardrail_function=reject_invalid_tickers)],
    handoffs=[compliance_handoff],
    handoff_description="Compliance_agent",
    model="gpt-4o-mini"
)

# Workflow Simulation
config = RunConfig(
    model_settings=ModelSettings(
        temperature=0.6
    )
)

async def main():
    output = []
    queries = ["Forecast NVDA price for the next 5 days."]
    for i, query in enumerate(queries):
        try:
            print(f'Q_{i+1}: {query}')
            result = await Runner.run(trading_assistant, query, run_config=config)
            print(f'A: {result.final_output}')  # Use final_output instead of result[0]['content']
            output.append(result)
        except Exception as e:
            if hasattr(e, 'guardrail_result'):
                reason = e.guardrail_result.output.output_info.reason
                print(f"Guardrail blocked input: {reason}")
            else:
                print(f"Error: {str(e)}")
            continue
        print("\n", 175 * "-", "\n")
    return queries, output

# Execute Workflow
if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    if loop.is_running():
        queries, result_list = await main()
    else:
        queries, result_list = asyncio.run(main())

Q_1: Forecast NVDA price for the next 5 days.
A: The forecast for NVIDIA Corporation (NVDA) was unsuccessful due to an error in retrieving the data. Please try again later or check another stock.

 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 

